In [13]:
import pandas as pd
import datetime as dt
import pycurl
from io import BytesIO

### Load Data ###

In [40]:
with open('data/source_data.csv', 'wb') as f:
    curl = pycurl.Curl()
    curl.setopt(curl.URL, "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
    curl.setopt(curl.WRITEDATA, f)
    curl.perform()
    curl.close()

covid_data = pd.read_csv("data/source_data.csv").rename(columns={'Country/Region': 'Country'})
covid_data = covid_data.drop(columns=['Province/State', 'Lat', 'Long'])
covid_data.head()

,Country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,22,24,24,40,40,74,84,94,110,110
1,Albania,0,0,0,0,0,0,0,0,0,...,64,70,76,89,104,123,146,174,186,197
2,Algeria,0,0,0,0,0,0,0,0,0,...,87,90,139,201,230,264,302,367,409,454
3,Andorra,0,0,0,0,0,0,0,0,0,...,53,75,88,113,133,164,188,224,267,308
4,Angola,0,0,0,0,0,0,0,0,0,...,0,1,2,2,3,3,3,4,4,5


### Scandinavia ###

In [41]:
covid_data_scandinavia = covid_data.query('Country == "Denmark" | Country == "Sweden" | Country == "Norway" | Country == "Finland"').drop([92, 93])
covid_data_scandinavia.to_csv('data/covid_data_scandinavia.csv')
covid_data_scandinavia.head()

,Country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20
94,Denmark,0,0,0,0,0,0,0,0,0,...,1151,1255,1326,1395,1450,1591,1724,1877,2046,2201
106,Finland,0,0,0,0,0,0,0,1,1,...,400,450,523,626,700,792,880,958,1041,1167
175,Norway,0,0,0,0,0,0,0,0,0,...,1746,1914,2118,2385,2621,2863,3084,3369,3755,4015
205,Sweden,0,0,0,0,0,0,0,0,0,...,1439,1639,1763,1934,2046,2286,2526,2840,3069,3447


### Reshape Data

In [45]:
# Melt dataframe and set index
def reshape_data(raw_df):
    cleaned_df = raw_df.melt(id_vars=['Country'], value_name='Cases', var_name='Date')
    cleaned_df = cleaned_df.set_index(['Country', 'Date'])
    return cleaned_df

# Clean dataset(s)
covid_data_scandinavia_reshaped = reshape_data(covid_data_scandinavia)
covid_data_scandinavia_reshaped.to_csv('data/covid_data_scandinavia_reshaped.csv')
covid_data_scandinavia_reshaped.head()

,,Cases
Country,Date,
Denmark,1/22/20,0
Finland,1/22/20,0
Norway,1/22/20,0
Sweden,1/22/20,0
Denmark,1/23/20,0


### Calculate and Merge Data

In [57]:
def country_data(cleaned_df, old_name, new_name):
    country_df = cleaned_df.groupby(['Country', 'Date'])['Cases'].sum().reset_index()
    country_df = country_df.set_index(['Country', 'Date'])
    country_df.index = country_df.index.set_levels([country_df.index.levels[0], pd.to_datetime(country_df.index.levels[1])])
    country_df = country_df.sort_values(['Country', 'Date'], ascending=True)
    country_df = country_df.rename(columns={old_name:new_name})
    return country_df
    
cases_country = country_data(covid_data_scandinavia_reshaped, 'Cases', 'Total Confirmed Cases')    

def daily_data(country_df, old_name, new_name):
    country_df_daily = country_df.groupby(level=0).diff().fillna(0)
    country_df_daily = country_df_daily.rename(columns={old_name:new_name})
    return country_df_daily

new_cases_country = daily_data(cases_country, 'Total Confirmed Cases', 'Daily New Cases')

merged_country_data = pd.merge(cases_country, new_cases_country, how='left', left_index=True, right_index=True)
merged_country_data

merged_country_data.to_csv('data/merged_data_scandinavia.csv')

### Long Form ###
Fits best with Altair charts.

In [5]:
dates = covid_data_scandinavia.columns.values.tolist()
dates.remove(dates[0])

long_form = covid_data_scandinavia.melt(id_vars=['Country'], value_vars=dates, var_name='Date', value_name='Cases')
long_form.to_csv('long_form_scandinavia.csv', index=False)

In [ ]:
dates = covid_data_scandinavia.columns.values.tolist()
dates.remove(dates[0])
denmark = covid_data_scandinavia.loc[94]
denmark = denmark.drop(denmark.index[0])
finland = covid_data_scandinavia.loc[106]
finland = finland.drop(finland.index[0])
norway = covid_data_scandinavia.loc[175]
norway = norway.drop(norway.index[0])
sweden = covid_data_scandinavia.loc[205]
sweden = sweden.drop(sweden.index[0])
data = {'Date': dates,
        'Denmark': denmark,
        'Finland': finland,
        'Norway': norway,
        'Sweden': sweden}

covid_data_scandinavia_reshaped = pd.DataFrame(data, columns=['Date', 'Denmark', 'Finland', 'Norway', 'Sweden']).set_index('Date')

In [131]:
covid_data_scandinavia_reshaped.to_csv('data/covid_data_scandinavia.csv')

### Denmark ###

In [74]:
covid_data_denmark = covid_data.query('Country == "Denmark"').drop([92, 93])
covid_data_denmark

,Country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20
94,Denmark,0,0,0,0,0,0,0,0,0,...,1057,1151,1255,1326,1395,1450,1591,1724,1877,2046


In [101]:
dates = covid_data_denmark.columns.values.tolist()
dates.remove(dates[0])
denmark = covid_data_denmark.loc[94]
denmark = denmark.drop(denmark.index[0])
data = {'Date': dates,
        'Denmark': denmark}

covid_data_denmark_reshaped = pd.DataFrame(data, columns=['Date', 'Denmark']).set_index('Date')
covid_data_denmark_reshaped.to_csv('denmark.csv')